In [1]:
import psycopg
import pandas as pd

In [2]:
host='postgresql.postgresql.svc.cluster.local'
dbname='stapp'
user = 'stapp'
password = 'stapp123'
port = '5432'

In [3]:
conn = psycopg.connect(
    f"host={host} dbname={dbname} user={user} password={password} port={port}"
) 

In [4]:
conn.autocommit = True

In [5]:
cursor = conn.cursor()

In [6]:
table_name = 'tickets'
CREATE_TABLE=f'CREATE TABLE {table_name} ( "ticketList_subject" varchar,"ticketList_detailproblem" varchar, "ticketList_source_cause" varchar(50), "ticketList_product_category" varchar(30), "ticketList_downtime" varchar(30),"ticketList_resolution" varchar, "ticketList_mttrall" real )'
cursor.execute(CREATE_TABLE)

<psycopg.Cursor [COMMAND_OK] [IDLE] (host=postgresql.postgresql.svc.cluster.local database=stapp) at 0x7f2794c7e560>

In [7]:
ticket_df = pd.read_csv('./test-tickets.csv',low_memory=False)
selected_column = ['ticketList_subject', 'ticketList_detailproblem','ticketList_source_cause','ticketList_product_category','ticketList_downtime','ticketList_resolution','ticketList_mttrall']
data = ticket_df[selected_column]

In [8]:
output_file = 'test_tickets_selected.csv'
data.to_csv(output_file, index=False)

In [9]:
COPY_CSV = f"COPY {table_name} FROM stdin (delimiter ',' , format csv, quote '\"', header) "
with open(output_file,'r') as file: 
    with cursor.copy(COPY_CSV) as copy:
        copy.write(file.read())

In [10]:
QUERY_TABLE = f'SELECT * FROM {table_name} LIMIT 10'
cursor.execute(QUERY_TABLE)

<psycopg.Cursor [TUPLES_OK] [IDLE] (host=postgresql.postgresql.svc.cluster.local database=stapp) at 0x7f2794c7e560>

In [11]:
datas = cursor.fetchall()
for row in datas:
    print(row)

('LINK TechnoSoft Solutions 100 MBPS SERVICE REQUEST', 'SERVICE REQUEST LIMITATION', 'FIBER OPTIC PROBLEM', 'NON GPON', '04/15/2024 02:45', 'Service restored after technical intervention', 30.01)
('LINK Digital Dynamics Inc 547 MBPS SERVICE REQUEST', 'No Problem at MTI side', 'SERVICE REQUEST', 'GPON', '03/29/2024 00:35', 'Service restored after technical intervention', 376.74)
('LINK ACME Corp Ltd 734 MBPS SERVICE REQUEST', 'No Problem at MTI side', 'SERVICE REQUEST', 'GPON', '04/12/2024 14:36', 'Service restored after technical intervention', 331.04)
('LINK Digital Dynamics Inc 778 MBPS SERVICE REQUEST', 'SERVICE REQUEST LIMITATION', 'SERVICE REQUEST', 'GPON', '04/09/2024 19:11', 'Service restored after technical intervention', 231.57)
('LINK TechnoSoft Solutions 487 MBPS SERVICE REQUEST', 'Link back to normal', 'EQUIPMENT PROBLEM', 'NON GPON', '04/08/2024 16:35', 'Service restored after technical intervention', 111.06)
('LINK ACME Corp Ltd 659 MBPS SERVICE REQUEST', 'Link back to no

In [12]:
cursor.execute('SELECT count(*) FROM tickets')

<psycopg.Cursor [TUPLES_OK] [IDLE] (host=postgresql.postgresql.svc.cluster.local database=stapp) at 0x7f2794c7e560>

In [13]:
cursor.fetchall()

[(50,)]

In [14]:
cursor.close()